<a href="https://colab.research.google.com/github/jegraham/Phase-1-Basic-GNN/blob/InMemoryDataset/Graham%2C_Jessica_Simple_GNN_Phase_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple GNN Implementation (Phase 1) 

This is the initial GNN implementation using encoders and decoders. 


## Import Libraries

In [3]:
import os
import shutil # -> For copying a directory 
import pandas as pd # -> For data structure and file manipulation

# libraries for the files in google drive
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials



In [4]:
#Import the remaining files as needed
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

# GPU Usage Guide - https://medium.com/@natsunoyuki/speeding-up-model-training-with-google-colab-b1ad7c48573e
if torch.cuda.is_available():
    device_name = torch.device("cuda")
else:
    device_name = torch.device('cpu')
print("Using {}.".format(device_name))

#!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
#print('uninstall')

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install --verbose torch-scatter
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install torch_geometric pandas
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{TORCH}.html

from torch_geometric.datasets import Planetoid
from torch_geometric.data import Data, InMemoryDataset
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import NormalizeFeatures, ToDevice, RandomLinkSplit
import torch.nn.functional as F
import math
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

2.0.1+cu118
Using cuda.
Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-{TORCH}.html


Check GPU

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Check Memory

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')
  

# Import the Dataset

Process the Data Frame - Modified Code from - https://github.com/jegraham/csv_to_dataframe_to_graph/blob/master/.idea/csv_to_datadrame_conversion.py

In [ ]:
# Import the Data
from google.colab import files 
uploaded = files.upload()
# uploaded = https://drive.google.com/file/d/1Ej2lijwlKM7pri5ebQ5hVl-Z5OtvWNbz/view?usp=share_link

file_name = next(iter(uploaded))
df = pd.read_csv(file_name)

#df.to_csv('/content/data.csv', index=False)
df = df.reset_index(drop=True)# Inspiration from Nazanin - https://github.com/nazaninmehregan/5P30-Final-Project/blob/master/5P30_Final_Project.ipynb

# Setup Dataset 


In [ ]:
# Generate a graph
# Inspirations taken from - Inspiration from Nazanin - https://github.com/nazaninmehregan/5P30-Final-Project/blob/master/5P30_Final_Project.ipynb

# Create an empty graph and lists
G = nx.Graph()
edge_index = []


# iterate through rows and add nodes to the graph
for i in range(len(df)):
    G.add_node(i, x=df.loc[i, 'X'], y=df.loc[i, 'Y'])


# Convert 'Car ID' column to numeric values
df['Car ID'] = pd.to_numeric(df['Car ID'], errors='coerce')

# Assign numeric IDs based on 'Car ID' column
car_ids = torch.tensor(df['Car ID'].values, dtype=torch.float)
vertex = car_ids  # Use 'Car ID' as the vertex tensor


# add nodes to the graph with their features 
for i, row in df.iterrows():
    G.add_node(i, **row.to_dict())

# Loop row by row to add the edges to the graph 
for i in range(len(df)):
   distances = []

   # Inner loop to iterate through all other vehicles
   for j in range(len(df)):
        if i == j:
            continue

        dist = math.sqrt((df.iloc[j]['X']-df.iloc[i]['X'])**2 + (df.iloc[j]['Y']-df.iloc[i]['Y'])**2)
        
        if 0 < dist <= 500:
          distances.append((dist, j))
        # print(distance)

   distances.sort()
        
   for k in range(len(distances)):
       G.add_edge(i, distances[k][1], weight=distances[k][0])

# print the graph
print(G.edges)
print(G)


In [ ]:
# Plotting Graph (Version 1)
X = df[['X', 'Y']].values

# create a layout for the graph nodes
pos = dict(zip(range(len(df)), X))

# draw the graph
fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(df['X'], df['Y'], s=20, color='grey')
nx.draw_networkx_nodes(G, pos, node_color='r', node_size=20, ax=ax)
nx.draw_networkx_edges(G, pos, edge_color='b', ax=ax)
ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.show()


In [ ]:
# Plotting Graph (Version 2)

# generate a layout for the graph nodes
pos = nx.spring_layout(G, k=1.1, iterations=50, seed=42)
fig, ax = plt.subplots(figsize=(15, 15))


# draw the nodes of the graph
nx.draw_networkx_nodes(G, pos, node_size=20, node_color='pink', alpha=0.7, edgecolors='pink')
nx.draw_networkx_labels(G, pos, font_size=10, font_family='sans-serif')

# draw the edges of the graph
nx.draw_networkx_edges(G, pos, edge_color='gray', alpha=0.5)

# get the edge weights as a dictionary
edge_labels = nx.get_edge_attributes(G, 'weight')

# draw the edge weights as labels
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)

# display the graph

plt.show()

In [ ]:
# Plotting Graph (Version 3)
# Displaying the Graph Edges Similar 

# specify node positions
pos = nx.spring_layout(G,k=1, iterations=50, seed=42) 
fig, ax = plt.subplots(figsize=(15, 15))

# draw nodes
nx.draw_networkx_nodes(G, pos) 

# draw node labels
nx.draw_networkx_labels(G, pos)

# draw edges
nx.draw_networkx_edges(G, pos)  


plt.show() 

print(G)

# CONVERT Dataset

Convert Dataset to same format as Planetoid - https://pytorch-geometric.readthedocs.io/en/latest/tutorial/create_dataset.html

In [121]:
#InMemoryDataset
class MyDataset(InMemoryDataset):
    def __init__(self, root, file_name, transform=None, pre_transform=None):
        self.filename = file_name     
        super(MyDataset, self).__init__(root, transform, pre_transform)
        #out = torch.load(self.processed_paths[0])
        
    # property
    # def raw_dir(self):
    #       print("raw")
    # #     return osp.join(self.root, self.name, 'raw')

    # property
    # def processed_dir(self):
    #      print("process dir")
    #      return osp.join(self.root, self.name, 'processed')

    @property
    def raw_file_names(self):
        print("1")
        return [self.filename]

    @property
    def processed_file_names(self):
       print("2")
       return ['data.pt']

    @property
    def donwload(self):
        print("3")
        df = pd.read_csv(self.root)
        df.to_csv(self.raw_paths[0], index=False)

    def read_file(self):
        print("4")
        df = pd.read_csv(self.filename)
        df = df.reset_index(drop=True)
        df['Type'] = np.where(df['Car ID'].str.contains('Tower'), 1, 0)
        #df['id'] = df[['Car ID']].sum(axis=1).map(hash)
        df = df.drop(columns=['Car ID'])
        print(df)
        return df

    def process(self):
        print('5')
        df = self.read_file()
        x = torch.from_numpy(df.values)     
        
        edge_source = []
        edge_target = []

        # Loop row by row to add the edges to the graph 
        for i in range(len(df)):
            # Inner loop to iterate through all other vehicles
            for j in range(len(df)):
                  if i == j:
                      continue

                  dist = math.sqrt((df.iloc[j]['X']-df.iloc[i]['X'])**2 + (df.iloc[j]['Y']-df.iloc[i]['Y'])**2)
                  
                  if 0 < dist <= 500:
                      edge_source.append(i)#No direction 
                      edge_target.append(j)
                      edge_source.append(j)
                      edge_target.append(i)

        print('source', edge_source)
        print('target', edge_target)

        # Create PyTorch Geometric Data object
        data = Data(
            x=x,  
            edge_index=torch.tensor([edge_source, edge_target])
       )
        

        print(self.processed_paths[0])
        print(data)
        self.save(data, self.processed_paths[0])   

# Define the root directory where the dataset will be stored
root = './'
file_name = 'May 4th Dataset Car and Tower  - Sheet1 (1).csv'
dest = os.path.join(root, file_name)

dataset = MyDataset(root,file_name)


print('Done')
# x = data['x']
# y = data['y']
# edge_index = data['edge_index']

# ### Display the graph here 
# # Convert the PyTorch Geometric data to a NetworkX graph
# G = nx.Graph()
# if i in data:
#         G.add_node(i, pos=(x[i].item(), y[i].item()))
# G.add_edges_from(edge_index.t().tolist())

# # Plot the graph based on x and y coordinates
# pos = nx.get_node_attributes(G, 'pos')
# plt.figure(figsize=(10, 10))
# nx.draw(G, pos, with_labels=True)
# plt.show()

2
5
4
               X            Y  Type                   id
0     900.000000   900.000000     0  4420293423330315448
1    3300.000000   900.000000     0 -3101921648116409877
2     900.000000  2500.000000     0  6288622709128213825
3    3300.000000  2500.000000     0 -5278189830759464413
4    1600.000000  1600.000000     0  6534988327034067196
..           ...          ...   ...                  ...
306   262.577682   943.846324     0 -3974465361292541074
307   480.406939  1938.562423     0 -3935877966296250801
308  2374.866031  2859.839327     0  6547408970849875285
309   965.998015  2617.177235     0   996689707527775057
310   415.717731  1289.906995     0 -6426935359963321320

[311 rows x 4 columns]


Processing...


source [0, 6, 0, 25, 0, 26, 0, 51, 0, 59, 0, 60, 0, 67, 0, 70, 0, 83, 0, 85, 0, 86, 0, 90, 0, 131, 0, 137, 0, 142, 0, 149, 0, 166, 0, 190, 0, 198, 0, 221, 0, 228, 0, 237, 0, 256, 0, 267, 0, 269, 0, 275, 0, 276, 0, 291, 1, 8, 1, 12, 1, 27, 1, 39, 1, 47, 1, 53, 1, 57, 1, 62, 1, 79, 1, 99, 1, 105, 1, 173, 1, 174, 1, 183, 1, 207, 1, 225, 1, 235, 1, 245, 1, 247, 1, 250, 2, 14, 2, 17, 2, 23, 2, 44, 2, 55, 2, 61, 2, 76, 2, 78, 2, 91, 2, 103, 2, 117, 2, 122, 2, 124, 2, 128, 2, 134, 2, 156, 2, 157, 2, 159, 2, 162, 2, 163, 2, 177, 2, 178, 2, 187, 2, 209, 2, 220, 2, 249, 2, 253, 2, 260, 2, 261, 2, 263, 2, 265, 2, 274, 2, 289, 2, 293, 2, 298, 2, 301, 2, 309, 3, 37, 3, 41, 3, 72, 3, 73, 3, 75, 3, 95, 3, 112, 3, 116, 3, 155, 3, 214, 3, 223, 3, 239, 3, 243, 3, 252, 3, 255, 3, 296, 3, 303, 4, 11, 4, 13, 4, 15, 4, 16, 4, 19, 4, 24, 4, 32, 4, 36, 4, 42, 4, 43, 4, 48, 4, 52, 4, 66, 4, 82, 4, 87, 4, 88, 4, 89, 4, 92, 4, 97, 4, 98, 4, 104, 4, 107, 4, 118, 4, 119, 4, 129, 4, 133, 4, 135, 4, 136, 4, 138, 4, 

KeyError: ignored

# Graph AutoEncoder GAE

Graph AutoEncoders GAE &  
Variational Graph Autoencoders VGAE    

[Tutorial 6 paper](https://arxiv.org/pdf/1611.07308.pdf)  
[Tutorial 6 code](https://github.com/rusty1s/pytorch_geometric/blob/master/examples/autoencoder.py)

### Load the data

In [90]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


transform = T.Compose([
    T.NormalizeFeatures(),
    T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True,
                      split_labels=True, add_negative_train_samples=True),
    T.ToDevice(device)])

transform = RandomLinkSplit(
    num_val=0.1, num_test=0.2, 
    is_undirected=True, 
    split_labels=True
    )

# My Dataset code
dataset = MyDataset(root, file_name, transform=T.NormalizeFeatures())
#dataset = Planetoid("\..", "CiteSeer", transform=T.NormalizeFeatures())
data = dataset[0]
#data.train_mask = data.val_mask = data.test_mask = None
# data = train_test_split_edges(data)
train_data, val_data, test_data = transform(data)

# Print information
print(dataset) 
print(train_data)
print('test', test_data)
print(data)
print('------------')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

train_pos_edge_index = train_data.pos_edge_label_index
print(train_pos_edge_index)
# data.train_mask = data.val_mask = data.test_mask = None
# data = train_test_split_edges(data)

CiteSeer()
Data(x=[3327, 3703], edge_index=[2, 6374], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327], pos_edge_label=[3187], pos_edge_label_index=[2, 3187], neg_edge_label=[3187], neg_edge_label_index=[2, 3187])
test Data(x=[3327, 3703], edge_index=[2, 7284], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327], pos_edge_label=[910], pos_edge_label_index=[2, 910], neg_edge_label=[910], neg_edge_label_index=[2, 910])
Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])
------------
Number of graphs: 1
Number of features: 3703
Number of classes: 6
tensor([[1799,  434,  435,  ..., 1724,  247,  559],
        [1811, 2361, 1875,  ..., 1958, 2063, 1013]])


### Define the Encoder

In [83]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv2 = GCNConv(2 * out_channels, out_channels, cached=True) # cached only for transductive learning

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)
    

### Define the Autoencoder

In [84]:
from torch_geometric.nn import GAE

In [85]:
# parameters
out_channels = 2
num_features = data.num_features #### RESUME - May 29th
epochs = 100

# model
model = GAE(GCNEncoder(num_features, out_channels))

# move to GPU (if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
data = dataset[0].to(device)
x = train_data.x.to(device)
train_pos_edge_index = train_data.pos_edge_label_index.to(device)

# inizialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [86]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)



In [91]:
for epoch in range(1, epochs + 1):
    loss = train()

    auc, ap = test(test_data.pos_edge_label_index, test_data.neg_edge_label_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

Data(x=[3327, 3703], edge_index=[2, 7284], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327], pos_edge_label=[910], pos_edge_label_index=[2, 910], neg_edge_label=[910], neg_edge_label_index=[2, 910])
Epoch: 001, AUC: 0.7115, AP: 0.7420
Epoch: 002, AUC: 0.7097, AP: 0.7417
Epoch: 003, AUC: 0.7071, AP: 0.7408
Epoch: 004, AUC: 0.7050, AP: 0.7401
Epoch: 005, AUC: 0.7029, AP: 0.7396
Epoch: 006, AUC: 0.7011, AP: 0.7392
Epoch: 007, AUC: 0.6994, AP: 0.7391
Epoch: 008, AUC: 0.6979, AP: 0.7394
Epoch: 009, AUC: 0.6962, AP: 0.7399
Epoch: 010, AUC: 0.6943, AP: 0.7405
Epoch: 011, AUC: 0.6921, AP: 0.7410
Epoch: 012, AUC: 0.6898, AP: 0.7412
Epoch: 013, AUC: 0.6875, AP: 0.7413
Epoch: 014, AUC: 0.6853, AP: 0.7413
Epoch: 015, AUC: 0.6833, AP: 0.7411
Epoch: 016, AUC: 0.6819, AP: 0.7410
Epoch: 017, AUC: 0.6807, AP: 0.7408
Epoch: 018, AUC: 0.6800, AP: 0.7406
Epoch: 019, AUC: 0.6796, AP: 0.7404
Epoch: 020, AUC: 0.6793, AP: 0.7402
Epoch: 021, AUC: 0.6794, AP: 0.7403
Epoch: 022, AUC: 0.6797, AP: 0.

In [93]:
Z = model.encode(x, train_pos_edge_index)
print(Z)

tensor([[ 0.2213,  0.2216],
        [-0.7549, -0.6576],
        [ 0.6247,  0.6100],
        ...,
        [-1.0857, -0.9986],
        [ 0.6247,  0.6100],
        [ 0.6247,  0.6100]], device='cuda:0', grad_fn=<AddBackward0>)


# K-Means

In [ ]:
import networkx as nx
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt


# Get the adjacency matrix of the graph
A = nx.adjacency_matrix(G).todense()

# Determine the optimal number of clusters using the elbow method
distortions = []
K = range(1, 10)
for k in K:
    kmeans = KMeans(n_clusters=k).fit(A)
    distortions.append(kmeans.inertia_)

# Plot the elbow curve
plt.plot(K, distortions, 'bx-')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')
plt.title('Elbow Method')
plt.show()

# Choose the optimal number of clusters based on the elbow curve
optimal_K = 2

# Run K-means clustering on the adjacency matrix
kmeans = KMeans(n_clusters=optimal_K).fit(A)

# Assign the nodes to the corresponding clusters
clusters = kmeans.labels_


colors = ["red" if c == 0 else "blue" if c == 1 else "green" for c in clusters]

# Plot the graph with the nodes colored based on the clusters
nx.draw(G, with_labels=True, node_color=colors)
plt.show()

# Spectral

In [ ]:
import numpy as np
import networkx as nx
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt


# Compute the Laplacian matrix
A = nx.to_numpy_array(G)
D = np.diag(np.sum(A, axis=1))
L = D - A

# Compute the eigenvalues and eigenvectors of the Laplacian matrix
eigenvalues, eigenvectors = np.linalg.eig(L)

# Sort the eigenvalues and corresponding eigenvectors in ascending order
idx = eigenvalues.argsort()
eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[:,idx]

# Use the first two eigenvectors to create the embedding matrix
embedding = np.column_stack((eigenvectors[:,0], eigenvectors[:,1]))

# Apply K-means clustering on the embedding matrix
kmeans = KMeans(n_clusters=2).fit(embedding)

# Assign the nodes to the corresponding clusters
clusters = kmeans.labels_

plt.figure(figsize=(8, 8))
pos = nx.spring_layout(G)
for i, cluster in enumerate(clusters):
    color = plt.cm.Set1(cluster / 4.)
    nx.draw_networkx_nodes(G, pos, [i], node_size=200, node_color=color)
nx.draw_networkx_edges(G, pos, alpha=0.5)
plt.show()